<a href="https://colab.research.google.com/github/messias077/REP/blob/main/Problema_Generalizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
""" Avaliando a generalização de algoritmos """

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine

""" Classe simplória para auxiliar na hora de rodar os modelos """
class Modelo():
    def __init__(self, modelo, descricao):
        self.modelo = modelo
        self.descricao = descricao
        self.acuracias = []  # Guarda a acurácia de cada rodada do modelo
    
    # Só para ficar mais simples na hora de chamar esses métodos
    def fit(self, X, y):
        self.modelo.fit(X, y)
    
    def predict(self, X):
        return self.modelo.predict(X)


# Carrega os dados
wine = load_wine()
X, y = wine.data, wine.target


def rodar_modelos(modelos, qtd_folds, qtd_rodadas):
    """
        Roda as folds diversas vezes (dependendo dos parâmetros passados) para
        cada modelo e acumula as acurácias para tirar a média no final das
        execuções.
    """
    for _ in range(qtd_rodadas):
        # Gera as folds
        kf = KFold(n_splits=qtd_folds)

        # Roda todas os n_splits do KFold
        for treinamento, teste in kf.split(X, y):
            X_treinamento = X[treinamento]
            y_treinamento = y[treinamento]

            X_teste = X[teste]
            y_teste = y[teste]

            # Roda os modelos
            for mod in modelos:
                mod.fit(X_treinamento, y_treinamento)
                y_pred = mod.predict(X_teste)
                mod.acuracias.append(accuracy_score(y_teste, y_pred))

# Guarda os modelos que serão rodados
modelos = []

# Definição dos modelos

# Logistic Regression convencional
modelos.append(Modelo(LogisticRegression(max_iter=10000), 'Logistic Regression - Convencional' ))

# Logistic Regression aninhado
lr_anin = Pipeline([
    ("padronizacao", StandardScaler()),
    ("classificador", LogisticRegression(max_iter=3000))
])
modelos.append(Modelo(lr_anin, 'Logistic Regression - Aninhado'))

# KNN convencional
modelos.append(Modelo(KNeighborsClassifier(), 'KNeighbors Classifier - Convencional'))

# KNN Aninhado
parametros = {'n_neighbors': [3, 5, 7]}
grid_search_knn = GridSearchCV(KNeighborsClassifier(), parametros, scoring='neg_mean_squared_error')
knn_anin = Pipeline([
    ("padronizacao", StandardScaler()),
    ("gsknn", grid_search_knn)
])
modelos.append(Modelo(knn_anin, 'KNeighbors Classifier - Aninhado'))

# Roda os modelos
rodar_modelos(modelos, 5, 10)

print("\n  --------------------------------------")
print("\n         **** Resultados ****")
print("\n  --------------------------------------")

for mod in modelos:
    print(f"\n * {mod.descricao}")
    print(f"   - Acurácia global..: {np.mean(mod.acuracias)}")
    print(f"   - Desvio padrão....: {np.std(mod.acuracias)}\n")



  --------------------------------------

         **** Resultados ****

  --------------------------------------

 * Logistic Regression - Convencional
   - Acurácia global..: 0.9385714285714286
   - Desvio padrão....: 0.0368089728016146


 * Logistic Regression - Aninhado
   - Acurácia global..: 0.9609523809523809
   - Desvio padrão....: 0.028267341226138717


 * KNeighbors Classifier - Convencional
   - Acurácia global..: 0.61
   - Desvio padrão....: 0.30984419722790923


 * KNeighbors Classifier - Aninhado
   - Acurácia global..: 0.8992063492063492
   - Desvio padrão....: 0.07152726867264826

